## Missionars and Cannibals - a lab activity on search strategies
---

## Prerequisites

- Python 3 (correctly installed)

- module `numpy`

- the AIMA libraries for search; two alternatives:
    - either (git-)clone the AIMA repository: <https://github.com/aimacode/aima-python>
    - or download from the course site the `search.py` file

- file `reporting.py`: contains some useful functions for inspecting what's going on

- file `utils.py`: it is used by some search implementations, simply place it in the same folder.

---


## The AIMA search library

The library provided with the textbook contains an implementation of the most notable search strategies.

To use them, two main steps:
1. **Decide a representation of a state**. A state can be represented with any object you like. However, in order to correctly use the Graph-search algorithms, the `__eq__()` method should be overrided correctly. *Suggestion: in the first exercise, opt to represent the state using a tuple. The `__eq__()` method iis already properly overridden, so it would be an easy way for starting with.*

2. **Define a new class**, that will inherit form the class `Problem`, and that will contain the methods and the attributes needed to apply the search strategies. In particular, our new subclass of `Problem` will have to implement/override:
    - constructor `def __init__(self, initial, goal=None)`:
        
        > *'''The constructor specifies the initial state, and possibly a goal state, if there is a unique goal. Your subclass's constructor can add other arguments.'''*
    
    - method `def actions(self, state):`
        
        > *'''Return the actions that can be executed in the given state. The result would typically be a list, but if there are many actions, consider yielding them one at a time in an iterator, rather than building them all at once.'''*
        
    - method `def result(self, state, action):`
        
        > *'''Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state).'''*

    - method `def goal_test(self, state):`
        
        > *'''Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough.'''*
        
    - method `def path_cost(self, c, state1, action, state2):`
        
        > *'''Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2. If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1.'''*
        
    - method `def h(self, node):`
		
        > *'''Returns the heuristic applied to the Node node'''*

## The U2 Problem

*(This problem was given as an exam the 16th of December 2005)*  
U2 are giving a concert in Dublin. **There are still 17 minutes.** Unfortunately, to reach the stage, the members of the band must cross a small, dark, and dangerous bridge... do not despair! They have a torch!!! (only one).

The bridge allows the passing of two persons at a time. The torch is mandatory to cross the bridge, and should be brought back and forth (cannot be thrown...). All the members are on the wrong side of the bridge, far from the stage...

Every member of the U2 walks at a different velocity, and they takes different time to cross the bridge: 
- Bono, **1 minute** 
- Edge, **2 minutes**
- Adam, **5 minutes**
- Larry, **10 minutes**
If two members cross the bridge together, it will take them the highest time to cross it (i.e., the faster will walk at the velocity of the slower).  
For example: if Bono and Larry will cross together, it will take them *10 minutes* to get over the bridge. If Larry brings back the torch, another 10 minutes will pass, and the mission will be failed.

**Suggestion**: To limit the search space, suppose that the members move always in couple in one direction, and only one member brings back the torch.

---

### About the heuristic

> - At most, only two persons can move at a time.
> - Let us group the members in couples.
> - Sort the member in descending order on the base of the crossing time, and:
>> - put in the first group the two slower members
>> - put in the second group the remaining members
>> - Every group will move at the velocity of the slowest in the group.
> - The heuristic function is the sum of the time required by each group still in the wrong side of the bridge.
> - Is this heuristic function admissible? Will it find the best solution?

---

### About the implementation

- Which state representation?

- Which actions?

- Which is the goal?

- Which path cost?


In [ ]:
# from pourproblem import *
from search import *
from reporting import *







class U2(Problem):
    '''How to represent the state?
    Choice: state as a tuple of tuples, indicating who is in which shore, and the presence of the torch
    Hence, the initial state is (("Bono", "Edge", "Adam", "Larry"), (), 1)
    The goal state will be ((), ("Bono", "Edge", "Adam", "Larry"), 0)'''

    # class variable
    timing = {
        "Bono": 1,
        "Edge": 2,
        "Adam": 5,
        "Larry": 10
    }
    
    # class method
    @classmethod
    def sortingf(cls, key):
        return cls.timing[key]
    
    
    
    def __init__(self, initial, goal):
        """Constructor"""
        # the tuples are always ordered in reverse order basing on the timing
        # this is useful since the first element of the tuple is always the slower one 
        initial = (tuple(sorted(initial[0], reverse=True, key=U2.sortingf)),
                        tuple(sorted(initial[1], reverse=True, key=U2.sortingf)),
                        initial[2])
        goal = (tuple(sorted(goal[0], reverse=True, key=U2.sortingf)),
                        tuple(sorted(goal[1], reverse=True, key=U2.sortingf)),
                        goal[2])
        super().__init__(initial, goal)

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        if len(action) == 2:
            return c + U2.timing[action[0]]
        else:
            return c + U2.timing[action]

    def actions(self, state):
        """The actions executable in this state."""
        if state[2]==1:
            pickUpFrom = state[0]
        else:
            pickUpFrom = state[1]
        result = [(first, second) for first in pickUpFrom
                  for second in pickUpFrom if first != second and U2.timing[first] > U2.timing[second]] +\
                 [single for single in pickUpFrom]
        return result

    def result(self, state, action):
        """The state that results from executing this action in this state."""
        if state[2]==1:
            source = list(state[0])
            dest = list(state[1])
        else:
            source = list(state[1])
            dest = list(state[0])
        if len(action) == 2:
            for x in action:
                source.remove(x)
                dest.append(x)
        else:
            source.remove(action)
            dest.append(action)
        source.sort(reverse=True, key=U2.sortingf)
        dest.sort(reverse=True, key=U2.sortingf)
        if state[2]:
            return (tuple(source), tuple(dest), 0)
        else:
            return (tuple(dest), tuple(source), 1)

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

        
    def h(self, node):
        thestate = node.state
        result = 0
        if len(thestate[0]) > 0:
            result += U2.timing[thestate[0][0]]
        if len(thestate[0]) > 2:
            result += U2.timing[thestate[0][2]]
        return result



u2=U2((("Bono", "Edge", "Adam", "Larry"), (), 1), ((), ("Bono", "Edge", "Adam", "Larry"), 0))
soln = astar_search(u2)
path = path_actions(soln)
print("Cost: ", soln.path_cost)
print(path)
path = path_states(soln)
print(path)

